## Synapse Table Generation
- Takes numpy synapse prediction files (6 integers) converts them into a pre-synaptic and a post-synaptic coordinate, and look up supervoxel IDs by chunk.
- The chunks are image chunks, not synapse chunks to minimize downloads per look up. 

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from cloudvolume import CloudVolume
import cloudvolume
import sys
if '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/' not in sys.path:
    sys.path.append('/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/transforms/')
    
from fanc_seg_utils.annotation_interface import rootID_lookup
from fanc_seg_utils.annotation_interface import neuroglancer_utilities

## Get prediction files
- This round was stored locally.
- Alternatively, use `synapse_download.py` to pull them from the GCS bucket. 

In [2]:
synapse_fname = '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/synapses/t1_synapse_SVIDs_v4.csv'
directory = '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/synapses/t1_synapses/8.6_8.6_45/'
p = Path(directory).glob('**/*')
files = [x for x in p if x.is_file()]
cv = CloudVolume(neuroglancer_utilities.get_cv_path('Dynamic_V4')['url'],use_https=True,agglomerate=False)


## Look up supervoxel IDs
- For each batch of coordinates, generate a new GSPointLoader object, format the coordinates, load them into the point_loader, and run the lookup. 

In [ ]:
rootID_lookup.init_table(synapse_fname)

for i in files:
    gs = rootID_lookup.GSPointLoader(cv)
    try:
        rootID_lookup.write_table(synapse_fname,i,gs)
    except:
        print('Fail')
        missed.append(i)

    

Segmentation IDs:   0%|          | 0/9 [00:00<?, ?it/s]

table created


Segmentation IDs:  88%|████████▊ | 7/8 [00:01<00:00,  6.16it/s]

In [ ]:
df = pd.read_csv(synapse_fname)


In [ ]:
missed = []
finished = df.source
for i in files:
    if not i.name in finished:
        links = rootID_lookup.get_links(i)
        if len(links) > 0:
            missed.append(i)

In [ ]:
len(missed)

In [ ]:
links = rootID_lookup.get_links(missed[0])
links_formatted = rootID_lookup.format_links(links)

In [ ]:
links_formatted

In [ ]:
links_formatted = '/Users/brandon/Documents/Repositories/Python/FANC_auto_recon/synapses/t1_synapse_roots_v4.csv'

for i in missed:
    write_table(table_name,i,gs)

In [ ]:
links = rootID_lookup.get_links(i)
flinks = rootID_lookup.format_links(links)
gs.add_points(flinks)


In [ ]:
seg_mip = cv.scale['resolution']
res = seg_mip / np.array([4.3,4.3,45])
pts_scaled = [pt // res for pt in flinks]
